<a href="https://colab.research.google.com/github/GGpark1/User-Log-data-analytics/blob/main/Collaborative_filltering_fin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 구매 기록이 한번도 없는 상품은 추천 목록에서 제외 -> 희소문제를 해결하기 위함 
  - 전자 제품 군의 14,950개의 상품 중에서 한번도 팔리지 않은 상품은 약 10,000개로 분석에 포함될 시 희소문제를 일으킬 가능성이 높음

- 편향 보정을 위해 스마트폰을 구매한 일부 비율을 뺄까?

## 구매 횟수 필터 없는 추천 시스템

In [1]:
from pandas.io import gbq

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#pd.options.display.float_format = "{:,.2f}".format
import warnings
warnings.simplefilter("ignore")

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [4]:
sql = """SELECT user_id, product_id, count(*) AS purchase_count
FROM empirical-weft-350709.CP2_ggpark.log_data
WHERE category_code LIKE 'electronics%' AND event_type = 'purchase'
GROUP BY user_id, product_id
ORDER BY purchase_count DESC"""
project_id = 'empirical-weft-350709'

df = pd.read_gbq(sql, project_id=project_id, dialect='standard')

In [5]:
df_ratings = df

In [6]:
df_ratings['purchase_count'].mean()

1.4209971178845675

In [7]:
sql = """SELECT product_id, brand, category_code FROM `empirical-weft-350709.CP2_ggpark.log_data`
WHERE event_type = 'purchase' AND category_code LIKE 'electronics%'
GROUP BY product_id, brand, category_code"""
project_id = 'empirical-weft-350709'

df_product = pd.read_gbq(sql, project_id=project_id, dialect='standard')

In [22]:
product_data = df_product
product_data.head()

,product_id,brand,category_code
0,11300053,texet,electronics.telephone
1,22400122,trust,electronics.audio.microphone
2,11300005,panasonic,electronics.telephone
3,9900453,byintek,electronics.video.projector
4,18500081,huion,electronics.tablet


In [9]:
df_fin = pd.pivot_table(df, index = ['user_id'], columns = ['product_id'], values = 'purchase_count')

In [10]:
df_fin

product_id,1000978,1001588,1002042,1002062,1002098,1002099,1002100,1002101,1002102,1002103,...,22400236,51600003,51600004,51600005,51600009,51600010,51600011,51600012,51600018,51600020
user_id,,,,,,,,,,,,,,,,,,,,,
303160429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
340041246,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
401021311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
403013066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410333972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566263549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
566265448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
566270060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
df_base_line = df_fin.fillna(1.42)

In [11]:
df_final = df_fin.fillna(0)

In [12]:
df_final['total'] = df_final.sum(axis=1)
df_filtered = df_final.loc[df_final['total'] > 1]
df_filtered.drop(columns='total', inplace = True)

df_filtered

In [16]:
matrix = df_filtered.values
user_ratings_mean = np.mean(matrix, axis = 1)

In [17]:
user_ratings_mean_reshape = user_ratings_mean.reshape(-1, 1)

In [18]:
matrix_user_mean = matrix - user_ratings_mean_reshape

In [19]:
pd.DataFrame(matrix_user_mean, columns = df_filtered.columns, index = df_filtered.index).head()

product_id,1000978,1001588,1002042,1002062,1002098,1002099,1002100,1002101,1002102,1002103,...,22400236,51600003,51600004,51600005,51600009,51600010,51600011,51600012,51600018,51600020
user_id,,,,,,,,,,,,,,,,,,,,,
403013066,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,...,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426
415873351,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,...,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426
427391662,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,...,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426
437371552,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,...,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426
446080337,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,...,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426


In [13]:
from scipy.sparse.linalg import svds

In [21]:
U, sigma, Vt = svds(matrix_user_mean, k = 12)

In [22]:
print(U.shape)
print(sigma.shape)
print(Vt.shape)

(72380, 12)
(12,)
(12, 4698)


In [23]:
sigma = np.diag(sigma)

In [24]:
sigma.shape

(12, 12)

In [25]:
sigma[0]

array([137.4352273,   0.       ,   0.       ,   0.       ,   0.       ,
         0.       ,   0.       ,   0.       ,   0.       ,   0.       ,
         0.       ,   0.       ])

In [26]:
dot_matrix = np.dot(U, sigma)

In [27]:
df_svd_predicted_purchase = np.dot(dot_matrix, Vt)

In [28]:
df_svd_predicted_purchase = df_svd_predicted_purchase + user_ratings_mean_reshape

In [29]:
df_svd_preds = pd.DataFrame(df_svd_predicted_purchase, columns = df_filtered.columns, index = df_filtered.index)
df_svd_preds.head()

product_id,1000978,1001588,1002042,1002062,1002098,1002099,1002100,1002101,1002102,1002103,...,22400236,51600003,51600004,51600005,51600009,51600010,51600011,51600012,51600018,51600020
user_id,,,,,,,,,,,,,,,,,,,,,
403013066,-0.000098,-0.000036,-0.000040,0.000240,0.000437,0.000424,0.000881,0.000010,0.000317,-0.000036,...,-0.000036,-0.000036,-0.000036,-0.000036,-0.000036,-0.000036,-0.000036,-0.000036,-0.000036,-0.000036
415873351,0.000414,0.000387,0.000386,0.000395,0.000406,0.000466,0.000404,0.000464,0.000480,0.000384,...,0.000384,0.000384,0.000384,0.000384,0.000384,0.000384,0.000384,0.000384,0.000384,0.000384
427391662,0.000002,0.000102,0.000091,0.000099,0.000081,-0.000102,0.000041,0.000337,0.000060,0.000094,...,0.000094,0.000094,0.000094,0.000094,0.000094,0.000094,0.000094,0.000094,0.000094,0.000094
437371552,0.000404,0.000397,0.000397,0.000408,0.000416,0.000438,0.000432,0.000421,0.000434,0.000397,...,0.000397,0.000397,0.000397,0.000397,0.000397,0.000397,0.000397,0.000397,0.000397,0.000397
446080337,0.000065,0.000085,0.000089,0.000083,0.000256,0.000721,0.000233,0.000345,0.000256,0.000091,...,0.000091,0.000091,0.000091,0.000091,0.000091,0.000091,0.000091,0.000091,0.000091,0.000091


In [20]:
def recommend_movies(df_svd_preds, user_id, ori_movies_df, ori_ratings_df, num_recommendations=5):
    
    #현재는 index로 적용이 되어있으므로 user_id - 1을 해야함.
    user_row_number = user_id 
    
    # 최종적으로 만든 pred_df에서 사용자 index에 따라 영화 데이터 정렬 -> 영화 평점이 높은 순으로 정렬 됌
    sorted_user_predictions = df_svd_preds.loc[user_row_number].sort_values(ascending=False)
    
    # 원본 평점 데이터에서 user id에 해당하는 데이터를 뽑아낸다. 
    user_data = ori_ratings_df[ori_ratings_df.user_id == user_id]
    
    # 위에서 뽑은 user_data와 원본 영화 데이터를 합친다. 
    user_history = user_data.merge(ori_movies_df, on = 'product_id').sort_values(['purchase_count'], ascending=False)
    
    # 원본 영화 데이터에서 사용자가 본 영화 데이터를 제외한 데이터를 추출
    recommendations = ori_movies_df[~ori_movies_df['product_id'].isin(user_history['product_id'])]
    # 사용자의 영화 평점이 높은 순으로 정렬된 데이터와 위 recommendations을 합친다. 
    recommendations = recommendations.merge( pd.DataFrame(sorted_user_predictions).reset_index(), on = 'product_id')
    # 컬럼 이름 바꾸고 정렬해서 return
    recommendations = recommendations.rename(columns = {user_row_number: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :]
                      

    return user_history, recommendations

In [31]:
already_rated, predictions = recommend_movies(df_svd_preds, 523974502, product_data, df_ratings, 10)

In [32]:
predictions

,product_id,brand,category_code,Predictions
2147,5100866,samsung,electronics.clocks,0.139439
1468,5100860,samsung,electronics.clocks,0.115693
1344,1005031,xiaomi,electronics.smartphone,0.085602
1476,1801739,samsung,electronics.video.tv,0.078664
1325,1004739,xiaomi,electronics.smartphone,0.059773
1462,5100577,samsung,electronics.clocks,0.049753
1460,5100564,samsung,electronics.clocks,0.032617
1205,5100722,huawei,electronics.clocks,0.026976
1464,5100719,samsung,electronics.clocks,0.026686
1534,1004836,samsung,electronics.smartphone,0.026021


In [25]:
from sklearn.metrics import mean_squared_error

In [26]:
def evaluate(sparse_matrix, pred_matrix):
        idx, jdx = sparse_matrix.to_numpy().nonzero()
        ys, preds = [], []
        for i, j in zip(idx, jdx):
            ys.append(sparse_matrix.iloc[i, j])
            preds.append(pred_matrix.iloc[i, j])

        error = mean_squared_error(ys, preds)
        return np.sqrt(error)

In [37]:
# Base Line -> 예측 데이터를 평균으로 채워 넣음
# Mean_squared_error
# 구매 횟수에 필터를 걸지 않은 결과

evaluate(df_filtered, df_base_line)

2.178659284777304

In [35]:
# 행렬 분해 방식으로 도출한 예측 데이터
# Mean_squared_error
# 구매 횟수에 필터를 걸지 않은 결과

evaluate(df_filtered, df_svd_preds)

2.07203304647919

## 구매 횟수 필터 있는 추천 시스템

In [4]:
sql = """SELECT user_id, product_id, count(*) AS purchase_count
FROM empirical-weft-350709.CP2_ggpark.log_data
WHERE category_code LIKE 'electronics%' AND event_type = 'purchase'
GROUP BY user_id, product_id
HAVING purchase_count <= 5
ORDER BY purchase_count DESC"""
project_id = 'empirical-weft-350709'

df_limit = pd.read_gbq(sql, project_id=project_id, dialect='standard')

In [29]:
df_limit.sort_values('purchase_count')

,user_id,product_id,purchase_count
146438,520100082,1005196,1
195242,527361074,1005197,1
195243,562094335,1004935,1
195244,516298516,1004892,1
195245,513835455,1005189,1
...,...,...,...
1609,559814370,1003311,5
1608,515405029,1005104,5
1607,514931143,1004237,5
1605,521656912,1002633,5


In [32]:
df_ratings_limit = df_limit
mean = df_ratings_limit['purchase_count'].mean()

In [6]:
df_fin_limit = pd.pivot_table(df_limit, index = ['user_id'], columns = ['product_id'], values = 'purchase_count')
df_fin_limit

product_id,1000978,1001588,1002042,1002062,1002098,1002099,1002100,1002101,1002102,1002103,...,22400236,51600003,51600004,51600005,51600009,51600010,51600011,51600012,51600018,51600020
user_id,,,,,,,,,,,,,,,,,,,,,
303160429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
340041246,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
401021311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
403013066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410333972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566263549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
566265448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
566270060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df_final_limit = df_fin_limit.fillna(0)

In [33]:
df_base_line_limit = df_fin_limit.fillna(mean)

In [9]:
df_final_limit['total'] = df_final_limit.sum(axis=1)
df_filtered_limit = df_final_limit.loc[df_final_limit['total'] > 1]
df_filtered_limit.drop(columns='total', inplace = True)

df_filtered_limit

product_id,1000978,1001588,1002042,1002062,1002098,1002099,1002100,1002101,1002102,1002103,...,22400236,51600003,51600004,51600005,51600009,51600010,51600011,51600012,51600018,51600020
user_id,,,,,,,,,,,,,,,,,,,,,
403013066,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
415873351,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
427391662,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
437371552,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
446080337,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566240992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
566243643,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
566249028,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
matrix_limit = df_filtered_limit.values
user_ratings_mean_limit = np.mean(matrix_limit, axis = 1)

In [11]:
user_ratings_mean_reshape_limit = user_ratings_mean_limit.reshape(-1, 1)

In [12]:
matrix_user_mean_limit = matrix_limit - user_ratings_mean_reshape_limit

In [14]:
U_limit, sigma_limit, Vt_limit = svds(matrix_user_mean_limit, k = 12)

In [15]:
sigma_limit = np.diag(sigma_limit)

In [16]:
dot_matrix_limit = np.dot(U_limit, sigma_limit)

In [17]:
df_svd_predicted_purchase_limit = np.dot(dot_matrix_limit, Vt_limit)

In [18]:
df_svd_predicted_purchase_limit = df_svd_predicted_purchase_limit + user_ratings_mean_reshape_limit

In [19]:
df_svd_preds_limit = pd.DataFrame(df_svd_predicted_purchase_limit, columns = df_filtered_limit.columns, index = df_filtered_limit.index)
df_svd_preds_limit.head()

product_id,1000978,1001588,1002042,1002062,1002098,1002099,1002100,1002101,1002102,1002103,...,22400236,51600003,51600004,51600005,51600009,51600010,51600011,51600012,51600018,51600020
user_id,,,,,,,,,,,,,,,,,,,,,
403013066,-0.000019,-0.000005,-0.000013,0.000526,0.000677,0.000977,0.000507,0.000589,0.000926,-0.000008,...,-0.000008,-0.000008,-0.000008,-0.000008,-0.000008,-0.000008,-0.000009,-0.000008,-0.000008,-0.000009
415873351,0.000337,0.000339,0.000336,0.000393,0.000419,0.000495,0.000370,0.000376,0.000425,0.000333,...,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333
427391662,0.000211,0.000134,0.000099,0.000195,0.000209,0.000380,0.000221,0.000656,0.000661,0.000106,...,0.000106,0.000106,0.000106,0.000106,0.000106,0.000106,0.000106,0.000106,0.000106,0.000106
437371552,0.000375,0.000371,0.000370,0.000412,0.000427,0.000491,0.000401,0.000418,0.000441,0.000370,...,0.000370,0.000370,0.000370,0.000370,0.000370,0.000370,0.000370,0.000370,0.000370,0.000370
446080337,0.000076,0.000095,0.000108,0.000001,-0.000034,0.001246,0.000368,0.000971,0.000259,0.000112,...,0.000112,0.000112,0.000112,0.000112,0.000112,0.000112,0.000112,0.000112,0.000112,0.000112


In [30]:
already_rated, predictions = recommend_movies(df_svd_preds_limit, 562094335, product_data, df_ratings_limit, 10)

In [31]:
predictions

,product_id,brand,category_code,Predictions
1543,1004833,samsung,electronics.smartphone,0.053320
1548,1004836,samsung,electronics.smartphone,0.047206
1545,1004856,samsung,electronics.smartphone,0.029971
1562,1004873,samsung,electronics.smartphone,0.028153
1542,1004750,samsung,electronics.smartphone,0.016125
1547,1004870,samsung,electronics.smartphone,0.015333
991,1002524,apple,electronics.smartphone,0.013902
1322,1004741,xiaomi,electronics.smartphone,0.007928
986,1002633,apple,electronics.smartphone,0.007364
1546,1004767,samsung,electronics.smartphone,0.005022


In [34]:
# Base Line -> 예측 데이터를 평균으로 채워 넣음
# Mean_squared_error
# 구매 횟수에 필터를 걸지 않은 결과

evaluate(df_filtered_limit, df_base_line_limit)

0.8815595103052039

In [27]:
# 행렬 분해 방식으로 도출한 예측 데이터
# Mean_squared_error
# 구매 횟수에 필터를 걸지 않은 결과

evaluate(df_filtered_limit, df_svd_preds_limit)

1.3497154677352388